In [ ]:
import torch
torch.__version__

from torch import nn

In [ ]:
import torchvision #contains datasets, model architecture, image transformation
from torchvision import datasets
from torchvision.transforms import ToTensor

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

In [ ]:
import matplotlib.pyplot as plt

# Check versions
# Note: your PyTorch version shouldn't be lower than 1.10.0 and torchvision version shouldn't be lower than 0.11
print(f"PyTorch version: {torch.__version__}\ntorchvision version: {torchvision.__version__}")


<li>root : where to download data to
    <li>train:bool : get train/test data
        <li>download:bool - download or nOt
            <li>transform = what transformation to apply
                

In [ ]:
#setup training _data

train_data = datasets.FashionMNIST(root = 'data',
                                   train = True,
                                   download = True,
                                   transform = ToTensor(),
                                   target_transform = None)

test_data = datasets.FashionMNIST(root = 'data',
                                  train = False,
                                  download = True,
                                  transform = ToTensor())
                                   

In [ ]:
train_data


In [ ]:
test_data

In [ ]:
train_data[0]

In [ ]:
image, label = train_data[0]
image, label

In [ ]:
len(train_data),len(train_data.targets),len(test_data)

In [ ]:
# See classes
class_names = train_data.classes
class_names

In [ ]:
import matplotlib.pyplot as plt
image, label = train_data[1]
print(f"Image shape: {image.shape}")
plt.imshow(image.squeeze(),cmap='gray') # image shape is [1, 28, 28] (colour channels, height, width)
plt.title(label);

In [ ]:
torch.randint(0, len(train_data),size=[1])

In [ ]:
torch.manual_seed(42)
fig = plt.figure(figsize=(9, 9))
rows, cols = 4, 4
for i in range(1, rows * cols + 1):
    random_idx = torch.randint(0, len(train_data), size=[1]).item()
    img, label = train_data[random_idx]
    fig.add_subplot(rows, cols, i)
    plt.imshow(img.squeeze(), cmap="gray")
    plt.title(class_names[label])
   # plt.axis(False);

# Prepare a dataloader
<li> help to load data into model
    <li>turns a large dataset into python iterbale of smaller chunks
        <li>forward and backward pass on huge dataset is imposiible so mini bathces are done 

In [ ]:
from torch.utils.data import DataLoader

In [ ]:
bs = 32 #bacthSize

#turn datasets into iterables
train_dataloader = DataLoader(train_data,batch_size = bs,shuffle = True)

test_dataloader = DataLoader(test_data,batch_size = bs)

print(f'dataloader : {train_dataloader , test_dataloader}')
print(f'Length of train dataloader :{len(train_dataloader)} batches of {bs}')
print(f'Length of test dataloader :{len(test_dataloader)} batches of {bs}')

In [ ]:
train_features_batch, train_labels_batch = next(iter(train_dataloader))
train_features_batch.shape, train_labels_batch.shape

In [ ]:
flatten = nn.Flatten()
flatten(train_features_batch[0]).shape

In [ ]:
class Fashion(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten  = nn.Flatten()
        self.layer_1 = nn.Linear(in_features = 784 ,out_features = 512)
        self.layer_2 = nn.Linear(in_features = 512 ,out_features = 10)
    
    def forward(self,x):
        return self.layer_2(self.layer_1(x))

model_0 = Fashion().to(device)    

In [ ]:
model_0

In [ ]:
def accuracy_fn(actual,pred):
    acc = torch.eq(actual,pred).sum().item()/len(actual)
    return acc

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(params = model_0.parameters(),lr=0.1)

In [ ]:
from tqdm.auto import tqdm


In [ ]:
class FashionMNISTModelV1(nn.Module):
    def __init__(self, input_shape: int, hidden_units: int, output_shape: int):
        super().__init__()
        self.layer_stack = nn.Sequential(
            nn.Flatten(), # flatten inputs into single vector
            nn.Linear(in_features=input_shape, out_features=hidden_units),
            nn.ReLU(),
            nn.Linear(in_features=hidden_units, out_features=output_shape),
            nn.ReLU()
        )
    
    def forward(self, x: torch.Tensor):
        return self.layer_stack(x)
    
torch.manual_seed(42)
model_1 = FashionMNISTModelV1(input_shape=784, # number of input features
    hidden_units=10,
    output_shape=len(class_names) # number of output classes desired
).to(device) # send model to GPU if it's available
next(model_1.parameters()).device # check model device    

In [ ]:
epochs =3
train_acc = 0
for epoch in tqdm(range(epochs)):
    print(f'EPoch : {epoch}')
    
    ###trainnig
    #loop through each training batches
    train_loss=0
    for batch,(X,y) in enumerate(train_dataloader):
        X = X.to(device)
        y = y.to(device)
        
        model_1.train()
        #print(X.shape)
        
        #forwrad pass
        y_pred = model_1(X)
        #print(y_pred)
       
        #calculate loss per batch
        loss = loss_fn(y_pred,y)
        #print(f'loss values {loss}')
        train_loss += loss
        
        train_acc += accuracy_fn(y,y_pred.argmax(dim=1)) # Go from logits -> pred labels
        print(f'train_acc{train_acc}')
        
        
        #optimizer zero grad
        optimizer.zero_grad()
        
        #loss backward
        loss.backward()
        
        #optimzer step
        optimizer.step()
        
        if batch%400 == 0 :
            print(f"Looked at {batch * len(X)}/{len(train_dataloader.dataset)} samples")
        
        train_loss /= len(train_dataloader)
    
    ### Testing
    # Setup variables for accumulatively adding up loss and accuracy 
    test_loss, test_acc = 0, 0 
    model_0.eval()
    with torch.no_grad():
        for X, y in test_dataloader:
            # 1. Forward pass
            test_pred = model_0(X)
           
            # 2. Calculate loss (accumatively)
            test_loss += loss_fn(test_pred, y) # accumulatively add up the loss per epoch

            # 3. Calculate accuracy (preds need to be same as y_true)
            test_acc += accuracy_fn(y, test_pred.argmax(dim=1))
        
        # Calculations on test metrics need to happen inside torch.inference_mode()
        # Divide total test loss by length of test dataloader (per batch)
        test_loss /= len(test_dataloader)

        # Divide total accuracy by length of test dataloader (per batch)
        test_acc /= len(test_dataloader)

    ## Print out what's happening
    print(f"\nTrain loss: {train_loss:.5f} | Test loss: {test_loss:.5f}, Test acc: {test_acc: .2f}%\n")

        

In [ ]:
#calcuate those predictions using the loss function and accuracy function.

torch.manual_seed(42)
def eval_model(model: torch.nn.Module, 
               data_loader: torch.utils.data.DataLoader, 
               loss_fn: torch.nn.Module, 
               accuracy_fn):
    """Returns a dictionary containing the results of model predicting on data_loader.

    Args:
        model (torch.nn.Module): A PyTorch model capable of making predictions on data_loader.
        data_loader (torch.utils.data.DataLoader): The target dataset to predict on.
        loss_fn (torch.nn.Module): The loss function of model.
        accuracy_fn: An accuracy function to compare the models predictions to the truth labels.

    Returns:
        (dict): Results of model making predictions on data_loader.
    """
    loss, acc = 0, 0
    model.eval()
    with torch.no_grad():
        for X, y in data_loader:
            X = X.to(device)
            y= y.to(device)
            # Make predictions with the model
            y_pred = model(X).to(device)
            
            # Accumulate the loss and accuracy values per batch
            loss += loss_fn(y_pred, y)
            acc += accuracy_fn(y, 
                                y_pred.argmax(dim=1)) # For accuracy, need the prediction labels (logits -> pred_prob -> pred_labels)
        
        # Scale loss and acc to find the average loss/acc per batch
        loss /= len(data_loader)
        acc /= len(data_loader)
        
    return {"model_name": model.__class__.__name__, # only works when model was created with a class
            "model_loss": loss.item(),
            "model_acc": acc}

# Calculate model 0 results on test dataset
model_0_results = eval_model(model=model_0, data_loader=test_dataloader,
    loss_fn=loss_fn, accuracy_fn=accuracy_fn
)
model_0_results

In [ ]:
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"
device

In [ ]:
class Fashion(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer_stack = nn.Sequential(
            nn.Flatten(),
            nn.Linear(in_features = 784 ,out_features = 10),
            nn.ReLU(),
            nn.Linear(in_features = 10 ,out_features = 10),
            nn.ReLU()
        )
        
    def forward(self,x):
        return self.layer_stack(x)

In [ ]:
torch.manual_seed(42)
model = Fashion().to(device)

next(model.parameters()).device

In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(params = model.parameters(),lr = 0.1)

In [ ]:
def train_loop(model,
               data_loader, 
               loss_fn,
               optimizer,
               accuracy_fn,
               device=device):
    
    train_loss = 0
    
    train_acc= 0
    
    for batch,(X, y) in enumerate(data_loader):
        
        X,y = X.to(device),y.to(device)
        
        #ForwardPass
        y_pred = model(X)  #shape will be [32,10] --> prob val for each class
        #print(f'shape of y_pred {y_pred.shape}')
        #print(f'shape of y {y.shape}')
        
        #loss function
        loss = loss_fn(y_pred,y) #[32,10] [10]
        #print(f'loss value {loss}')
        train_loss += loss
        
        accuracy = accuracy_fn(y_pred.argmax(dim=1),y)
        train_acc +=accuracy
        
        
        #zero grad
        optimizer.zero_grad()
        
        #backpropogate
        loss.backward()
        
        #optimizeer Step
        optimizer.step()
        
    train_loss /=len(data_loader)    
    train_acc /= len(data_loader)
    print(f"Train loss: {train_loss:.5f} | Train accuracy: {train_acc:.2f}%")


def test_loop(model,data_loader,loss_fn,accuracy_fn,device=device)    :
    
    test_acc = 0
    test_loss= 0
    model.eval()
    
    with torch.no_grad():
        for X,y in data_loader:
            X,y = X.to(device),y.to(device)
            test_pred = model(X)
            
            # 2. Calculate loss and accuracy
            test_loss += loss_fn(test_pred, y)
            test_acc += accuracy_fn(y,test_pred.argmax(dim=1) # Go from logits -> pred labels
            )
        test_loss /= len(data_loader)
        test_acc /= len(data_loader)
        print(f"Test loss: {test_loss:.5f} | Test accuracy: {test_acc:.2f}%\n")    


In [ ]:
epochs = 3
for epoch in tqdm(range(epochs)):
    print(f"Epoch: {epoch}\n---------")
    train_loop(data_loader=train_dataloader, 
        model=model, 
        loss_fn=loss_fn,
        optimizer=optimizer,
        accuracy_fn=accuracy_fn
    )
    test_loop(data_loader=test_dataloader,
        model=model,
        loss_fn=loss_fn,
        accuracy_fn=accuracy_fn
    )

In [ ]:


# Move values to device
torch.manual_seed(42)
def eval_model(model: torch.nn.Module, 
               data_loader: torch.utils.data.DataLoader, 
               loss_fn: torch.nn.Module, 
               accuracy_fn, 
               device: torch.device = device):
    """Evaluates a given model on a given dataset.

    Args:
        model (torch.nn.Module): A PyTorch model capable of making predictions on data_loader.
        data_loader (torch.utils.data.DataLoader): The target dataset to predict on.
        loss_fn (torch.nn.Module): The loss function of model.
        accuracy_fn: An accuracy function to compare the models predictions to the truth labels.
        device (str, optional): Target device to compute on. Defaults to device.

    Returns:
        (dict): Results of model making predictions on data_loader.
    """
    loss, acc = 0, 0
    model.eval()
    with torch.no_grad():
        for X, y in data_loader:
            # Send data to the target device
            X, y = X.to(device), y.to(device)
            y_pred = model(X)
            loss += loss_fn(y_pred, y)
            acc += accuracy_fn(y, y_pred.argmax(dim=1))
        
        # Scale loss and acc
        loss /= len(data_loader)
        acc /= len(data_loader)
    return {"model_name": model.__class__.__name__, # only works when model was created with a class
            "model_loss": loss.item(),
            "model_acc": acc}
model_1_results = eval_model(model=model, data_loader=test_dataloader,
    loss_fn=loss_fn, accuracy_fn=accuracy_fn,
    device=device
)
model_1_results

# CNN

In [ ]:
class Fashion(nn.Module):
    
    def __init__(self,input_shape,hidden_units,out_shape):
        super().__init__()
        self.block_1 = nn.Sequential(                
            nn.Conv2d(in_channels = input_shape,
                 out_channels = hidden_units,
                 kernel_size = 3,
                 stride = 1,
                 padding = 1
                     ),
            nn.ReLU(),
            nn.Conv2d(in_channels = hidden_units,
                 out_channels = hidden_units,
                 kernel_size = 3,
                 stride = 1,
                 padding = 1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2,stride = 2))
                        
        self.block_2 = nn.Sequential(
            nn.Conv2d(hidden_units, hidden_units, 3, padding=1),
            nn.ReLU(),
            nn.Conv2d(hidden_units, hidden_units, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(in_features=hidden_units*7*7,out_features=out_shape)
        )
        
    def forward(self,x):
        x = self.block_1(x)
        print(x.shape)
        x = self.block_2(x)
        print(x.shape)
        x = self.classifier(x)
        
        return x
        

In [ ]:
torch.manual_seed(42)
model_2 = Fashion(input_shape=1,hidden_units=10,out_shape=10).to(device)
model_2

In [ ]:
images = torch.randn(size=(32, 3, 64, 64)) # [batch_size, color_channels, height, width]
test_image = images[0] # get a single image for testing
print(f"Image batch shape: {images.shape} -> [batch_size, color_channels, height, width]")
print(f"Single image shape: {test_image.shape} -> [color_channels, height, width]") 
print(f"Single image pixel values:\n{test_image}")

In [ ]:
conv_layer = nn.Conv2d(in_channels=3,
                       out_channels=10,
                       kernel_size=3,
                       stride=1,
                       padding=0) # also try using "valid" or "same" here 

# Pass the data through the convolutional layer
conv_layer(test_image)

In [ ]:
conv_layer(test_image.unsqueeze(dim=0))

In [7]:
from torchtext import data

In [5]:
import spacy


In [8]:
import spacy
LABEL = data.LabelField()
nlp = spacy.load("en_core_web_sm")


AttributeError: module 'spacy' has no attribute 'load'

In [ ]:

TWEET = data.Field(tokenize = 'spacy',lower = True)

In [ ]:
nlp = spacy.load('en')

# 